## Import libraries

In [1]:
import gc
import pickle
import optuna
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Model, load_model
from tensorflow_addons.optimizers import AdamW, Lookahead

## Load autoencoder model

In [2]:
autoencoder = load_model('../input/customer-churn-rate-dae/DAE_model.h5')
feature_model = Model(inputs=autoencoder.input,
                      outputs=autoencoder.get_layer('Embedding').output)
feature_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder-Input (InputLayer)      [(None, 142)]        0                                            
__________________________________________________________________________________________________
Encoder-Dense-1 (Dense)         (None, 128)          18304       Encoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-BatchNorm-1 (BatchNorma (None, 128)          512         Encoder-Dense-1[0][0]            
__________________________________________________________________________________________________
Encoder-Swish-1 (Activation)    (None, 128)          0           Encoder-BatchNorm-1[0][0]        
______________________________________________________________________________________________

## Prepare data for model training

In [3]:
with open("../input/customer-churn-rate-preprocess-data/Churn_Risk_Rate_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

302

In [4]:
cat_cols = ['region_category','membership_category','preferred_offer_types','internet_option',
            'complaint_status','feedback','age_category','joining_dt_year','time_category',
            'complaint_category','feedback_category','last_login_category']

train_df1, test_df1 = train_df.copy(), test_df.copy()

for col in cat_cols:
    dummy_val = pd.get_dummies(train_df1[col], prefix='col')
    train_df1 = pd.concat([train_df1, dummy_val], axis=1)
    train_df1.drop([col], inplace=True, axis=1)

print("train_df1: {}".format(train_df1.shape))

for col in cat_cols:
    dummy_val = pd.get_dummies(test_df1[col], prefix='col')
    test_df1 = pd.concat([test_df1, dummy_val], axis=1)
    test_df1.drop([col], inplace=True, axis=1)

print("test_df1: {}".format(test_df1.shape))

train_df1: (36987, 143)
test_df1: (19919, 142)


In [5]:
Xtrain_embed = feature_model.predict(train_df1.loc[:, train_df1.columns != 'churn_risk_score'].values)
Xtest_embed = feature_model.predict(test_df1.values)
Xtrain_embed_df = pd.DataFrame(Xtrain_embed, index=train_df1.index)
Xtest_embed_df = pd.DataFrame(Xtest_embed, index=test_df1.index)
print("Xtrain_embed_df: {} \nXtest_embed_df: {}".format(Xtrain_embed_df.shape, Xtest_embed_df.shape))

train_df = pd.merge(train_df, Xtrain_embed_df, on='customer_id', sort=False)
test_df = pd.merge(test_df, Xtest_embed_df, on='customer_id', sort=False)
print("train_df: {} \ntest_df: {}".format(train_df.shape, test_df.shape))

del train_df1
del test_df1
del Xtrain_embed
del Xtest_embed
del Xtrain_embed_df
del Xtest_embed_df
gc.collect()

Xtrain_embed_df: (36987, 64) 
Xtest_embed_df: (19919, 64)
train_df: (36987, 170) 
test_df: (19919, 169)


1393

In [6]:
train_df['churn_risk_score'] = train_df['churn_risk_score'].apply(lambda x: 0 if x==5 else 1 if x==-1 else x)
train_df.groupby(['churn_risk_score']).size().reset_index().rename(columns={0:'count'})

,churn_risk_score,count
0,0,9825
1,1,3815
2,2,2741
3,3,10422
4,4,10184


In [7]:
cat_cols = ['gender','joined_through_referral','used_special_discount','offer_application_preference',
            'past_complaint','joining_dt_day_weekend','referral_id_present','has_desktop','has_smartphone',
            'region_category','membership_category','preferred_offer_types','internet_option','complaint_status',
            'feedback','age_category','joining_dt_year','time_category','complaint_category','feedback_category',
            'last_login_category']

train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[1, 4, 12, 13, 14, 19, 32, 37, 38, 2, 3, 5, 6, 15, 16, 17, 18, 33, 34, 35, 36]


In [8]:
Xtrain = train_df.loc[:, train_df.columns != 'churn_risk_score'].copy()
Ytrain = train_df['churn_risk_score'].copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

Xtrain: (36987, 169) 
Ytrain: (36987,) 
Xtest: (19919, 169)


In [9]:
temp_df = train_df.groupby(['churn_risk_score']).size().reset_index().rename(columns={0:'count'})
total_count = np.sum(temp_df['count'].values)
temp_df['class%'] = (temp_df['count'] / total_count) * 100
lowest_pct = min(temp_df['class%'])
temp_df['class_weight'] = lowest_pct / temp_df['class%']
class_weight = temp_df[['churn_risk_score', 'class_weight']].to_dict()['class_weight']
class_weight

{0: 0.2789821882951654,
 1: 0.7184796854521627,
 2: 1.0,
 3: 0.26300134331222413,
 4: 0.2691476826394344}

## Hyperparameters search using Optuna

In [10]:
'''
def objective(trial):

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    counter = 0
    oof_score = 0

    for train, val in kfold.split(Xtrain.values, Ytrain.values):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            class_weights=class_weight,
            learning_rate=0.075,
            reg_lambda=trial.suggest_loguniform("reg_lambda", 1e-4, 1.0),
            bootstrap_type='Poisson',
            subsample=trial.suggest_uniform("subsample", 0.5, 1.0),
            max_depth=trial.suggest_int("max_depth", 7, 16), 
            grow_policy='Lossguide',
            min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 15), 
            max_leaves=trial.suggest_int("max_leaves", 50, 2000),
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=False)
        y_pred = model.predict(val_x)
        oof_score += 100 * f1_score(val_y, y_pred, average='macro')
    
    oof_score /= float(counter)
    return oof_score
'''

'\ndef objective(trial):\n\n    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)\n    counter = 0\n    oof_score = 0\n\n    for train, val in kfold.split(Xtrain.values, Ytrain.values):\n        counter += 1\n\n        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]\n        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]\n\n        model = CatBoostClassifier(\n            objective=\'MultiClass\',\n            eval_metric=\'MultiClass\',\n            num_boost_round=5000,\n            class_weights=class_weight,\n            learning_rate=0.075,\n            reg_lambda=trial.suggest_loguniform("reg_lambda", 1e-4, 1.0),\n            bootstrap_type=\'Poisson\',\n            subsample=trial.suggest_uniform("subsample", 0.5, 1.0),\n            max_depth=trial.suggest_int("max_depth", 7, 16), \n            grow_policy=\'Lossguide\',\n            min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 15), \n            max_leaves=trial.suggest_int("max_leaves"

In [11]:
'''
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
'''

"\nstudy = optuna.create_study(direction='maximize')\nstudy.optimize(objective, n_trials=100)\n"

In [12]:
'''
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("Value: {}".format(trial.value))

print("Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key, value))
'''

'\nprint("Number of finished trials: {}".format(len(study.trials)))\n\nprint("Best trial:")\ntrial = study.best_trial\n\nprint("Value: {}".format(trial.value))\n\nprint("Params: ")\nfor key, value in trial.params.items():\n    print(" {}: {}".format(key, value))\n'

## Build and validate the model

In [13]:
FOLD = 7
NUM_SEED = 5

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], 5))
y_pred_final_cb = np.zeros((Xtest.shape[0], 5))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            class_weights=class_weight,
            learning_rate=0.07964,
            reg_lambda=0.95,
            bootstrap_type='Poisson',
            subsample=0.81,
            max_depth=7, 
            grow_policy='Lossguide',
            min_data_in_leaf=10, 
            max_leaves=152,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=100)

        y_pred = model.predict(val_x)
        y_pred_meta_cb[val] += model.predict_proba(val_x)
        y_pred_final_cb += model.predict_proba(Xtest)
        
        score = 100 * f1_score(val_y, y_pred, average='macro')
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_cb = y_pred_meta_cb / float(NUM_SEED)
y_pred_final_cb = y_pred_final_cb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 1.4311264	test: 1.4358972	best: 1.4358972 (0)	total: 97.1ms	remaining: 8m 5s
100:	learn: 0.4348872	test: 0.5690128	best: 0.5681166 (82)	total: 4.13s	remaining: 3m 20s
200:	learn: 0.3634371	test: 0.5731835	best: 0.5681166 (82)	total: 7.87s	remaining: 3m 7s
bestTest = 0.5681166189
bestIteration = 82
Shrink model to first 83 iterations.
Seed-24 | Fold-0 | OOF Score: 72.2076768307455


0:	learn: 1.4323463	test: 1.4336982	best: 1.4336982 (0)	total: 56.1ms	remaining: 4m 40s
100:	learn: 0.4336165	test: 0.5624492	best: 0.5621323 (89)	total: 4.31s	remaining: 3m 29s
200:	learn: 0.3627438	test: 0.5623917	best: 0.5611208 (152)	total: 8.5s	remaining: 3m 23s
300:	learn: 0.3058763	test: 0.5662300	best: 0.5611208 (152)	total: 13.7s	remaining: 3m 33s
bestTest = 0.5611207963
bestIteration = 152
Shrink model to first 153 iterations.
Seed-24 | Fold-1 | OOF Score: 72.91464103802619


0:	learn: 1.4314860	test: 1.4335520	best: 1.4335520 (0)	total: 52.9ms	remaining: 4m 24s
100:	learn: 0.4361763	test: 0.5576705	best: 0.5571326 (86)	total: 4.69s	remaining: 3m 47s
200:	learn: 0.3652428	test: 0.5589103	best: 0.5566511 (107)	total: 8.44s	remaining: 3m 21s
300:	learn: 0.3115595	test: 0.5632189	best: 0.5566511 (107)	total: 12.8s	remaining: 3m 19s
bestTest = 0.5566510757
bestIteration = 107
Shrink model to first 108 iterations.
Seed-24 | Fold-2 | OOF Score: 73.23759101715265


0:	learn: 1.4311574	test: 1.4335921	best: 1.4335921 (0)	total: 55.2ms	remaining: 4m 36s
100:	learn: 0.4313804	test: 0.5582578	best: 0.5582578 (100)	total: 4.24s	remaining: 3m 25s
200:	learn: 0.3587709	test: 0.5616041	best: 0.5578264 (121)	total: 8.22s	remaining: 3m 16s
300:	learn: 0.3026800	test: 0.5650809	best: 0.5578264 (121)	total: 13.2s	remaining: 3m 25s
bestTest = 0.5578263882
bestIteration = 121
Shrink model to first 122 iterations.
Seed-24 | Fold-3 | OOF Score: 72.66449599634626


0:	learn: 1.4311823	test: 1.4345928	best: 1.4345928 (0)	total: 51.1ms	remaining: 4m 15s
100:	learn: 0.4353306	test: 0.5568694	best: 0.5563466 (89)	total: 4.72s	remaining: 3m 48s
200:	learn: 0.3668084	test: 0.5597136	best: 0.5562718 (133)	total: 8.32s	remaining: 3m 18s
300:	learn: 0.3102801	test: 0.5647263	best: 0.5562718 (133)	total: 12.3s	remaining: 3m 11s
bestTest = 0.5562718053
bestIteration = 133
Shrink model to first 134 iterations.
Seed-24 | Fold-4 | OOF Score: 73.26273915344981


...

0:	learn: 1.4315993	test: 1.4350382	best: 1.4350382 (0)	total: 52ms	remaining: 4m 20s
100:	learn: 0.4308615	test: 0.5596073	best: 0.5592802 (91)	total: 4.45s	remaining: 3m 35s
200:	learn: 0.3655053	test: 0.5595013	best: 0.5577763 (150)	total: 9.29s	remaining: 3m 41s
300:	learn: 0.3092579	test: 0.5627224	best: 0.5577763 (150)	total: 13.5s	remaining: 3m 31s
bestTest = 0.5577763033
bestIteration = 150
Shrink model to first 151 iterations.
Seed-0 | Fold-2 | OOF Score: 73.07382761149161


0:	learn: 1.4321834	test: 1.4337962	best: 1.4337962 (0)	total: 53.6ms	remaining: 4m 28s
100:	learn: 0.4359459	test: 0.5478941	best: 0.5475108 (97)	total: 4.27s	remaining: 3m 27s
200:	learn: 0.3621337	test: 0.5518086	best: 0.5475108 (97)	total: 8.08s	remaining: 3m 12s
bestTest = 0.5475108304
bestIteration = 97
Shrink model to first 98 iterations.
Seed-0 | Fold-3 | OOF Score: 73.66194271071856


0:	learn: 1.4319805	test: 1.4317275	best: 1.4317275 (0)	total: 52.6ms	remaining: 4m 22s
100:	learn: 0.4343653	test: 0.5471680	best: 0.5461976 (84)	total: 4.68s	remaining: 3m 47s
200:	learn: 0.3647875	test: 0.5495385	best: 0.5461976 (84)	total: 9.72s	remaining: 3m 52s
bestTest = 0.5461975813
bestIteration = 84
Shrink model to first 85 iterations.
Seed-0 | Fold-4 | OOF Score: 73.44463715569961


0:	learn: 1.4315251	test: 1.4339110	best: 1.4339110 (0)	total: 56.7ms	remaining: 4m 43s
100:	learn: 0.4365039	test: 0.5620790	best: 0.5618524 (74)	total: 4.6s	remaining: 3m 43s
200:	learn: 0.3627241	test: 0.5653401	best: 0.5616403 (118)	total: 8.43s	remaining: 3m 21s
300:	learn: 0.3086669	test: 0.5705097	best: 0.5616403 (118)	total: 12.4s	remaining: 3m 13s
bestTest = 0.5616402691
bestIteration = 118
Shrink model to first 119 iterations.
Seed-0 | Fold-5 | OOF Score: 72.39127006839958


0:	learn: 1.4306597	test: 1.4353142	best: 1.4353142 (0)	total: 55ms	remaining: 4m 35s
100:	learn: 0.4347986	test: 0.5770535	best: 0.5762758 (83)	total: 4.45s	remaining: 3m 35s
200:	learn: 0.3647363	test: 0.5807941	best: 0.5762758 (83)	total: 8.55s	remaining: 3m 24s
bestTest = 0.5762758208
bestIteration = 83
Shrink model to first 84 iterations.
Seed-0 | Fold-6 | OOF Score: 71.28595641106543

Seed: 0 | Aggregate OOF Score: 72.62402923513777


Aggregate OOF Score: 73.00284261974838


In [14]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

## Create submission file

In [15]:
test_df = pd.read_csv("../input/customer-churn-rate-prediction/dataset/test.csv")
submit_df = pd.DataFrame()
submit_df['customer_id'] = test_df['customer_id']
submit_df['churn_risk_score'] = np.array([np.argmax(y_pred_final_cb, axis=1)]).T
submit_df['churn_risk_score'] = submit_df['churn_risk_score'].apply(lambda x: 5 if x==0 else x)
submit_df.head()

,customer_id,churn_risk_score
0,fffe43004900440031003700300030003400,3
1,fffe43004900440031003900370037003300,3
2,fffe43004900440034003800360037003000,3
3,fffe43004900440036003200370033003400,3
4,fffe43004900440035003000370031003900,5


In [16]:
submit_df.to_csv("./CB_submission.csv", index=False)